In [ ]:
!pip install openai
!pip install diffusers transformers accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import openai
import time
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
from transformers import AutoTokenizer
from PIL import Image
from google.colab import files
import torch

In [ ]:
# HuggingFace 로그인 (토큰 입력 필요)
from huggingface_hub import notebook_login

notebook_login()  # 여기서 발급받은 토큰 입력

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="${API-KEY}")  # 본인 API 키를 넣으세요

def ask_gpt(messages):
    response = client.chat.completions.create(
        model="gpt-4o",  # 모델은 원하시는 걸로 변경
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
# ====== ControlNet 로드
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-scribble", torch_dtype=torch.float16
)

# ====== Stable Diffusion 파이프라인 로드 + ControlNet 결합
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "Lykon/dreamshaper-8",
    controlnet=controlnet,
    torch_dtype=torch.float16
).to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# === 줄거리 기반 그림 생성 함수
def generate_image(story_text, scene_number):

    # 프롬프트 생성
    image_prompt = f"""
    다음 동화 내용을 기반으로 영어로 AI 그림 프롬프트를 만들어줘.
    anime style, masterpiece, pastel color, soft light 포함

    {story_text}

    [AI 이미지 프롬프트]
    """

    messages.append({"role": "user", "content": image_prompt})
    ai_prompt = ask_gpt(messages)

    print(f"\n🎨 [장면 {scene_number} 프롬프트]\n", ai_prompt)

    # 아이 그림 업로드
    print("아이 그림을 업로드하세요.")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    scribble_image = load_image(filename).resize((512, 512))

    # ControlNet 이미지 생성
    output = pipe(
        prompt=ai_prompt,
        negative_prompt="lowres, bad anatomy, blurry, ugly, bad hands, extra fingers, poorly drawn, nsfw",
        image=scribble_image,
        num_inference_steps=60,
        guidance_scale=12.5,
        controlnet_conditioning_scale=0.8
    )

    result = output.images[0]
    result.show()

    # 저장
    save_name = f"storybook_scene_{scene_number}.png"
    result.save(save_name)
    print(f"✅ 저장 완료 → {save_name}")

In [ ]:
# ================================================
# ✅ 1. 아이 + 부모가 주인공 정보 직접 입력
# ================================================

character_name = input("주인공 이름: ")
character_species = input("주인공 종류: ")
character_age = input("주인공 나이: ")
character_personality = input("주인공 성격: ")
character_feature = input("주인공 특징: ")

character_info = f"""
[주인공 캐릭터]
이름: {character_name}
종류: {character_species}
나이: {character_age}
성격: {character_personality}
특징: {character_feature}
"""

print("📌 [주인공 설정 완료]\n", character_info)

story_log = []
story_log.append("[주인공 설정]\n" + character_info)

주인공 이름: 도레미
주인공 종류: 용사
주인공 나이: 8
주인공 성격: 용감한, 정의로운
주인공 특징: 모험을 좋아함
📌 [주인공 설정 완료]
 
[주인공 캐릭터]
이름: 도레미
종류: 용사
나이: 8
성격: 용감한, 정의로운
특징: 모험을 좋아함



In [ ]:
# ================================================
# ✅ 2. 동화 시작 줄거리 생성
# ================================================

story_start_prompt = f"""
아래 주인공을 바탕으로 동화의 시작 부분을 3~5문장으로 써줘.

{character_info}

[동화 시작]
"""

messages = [{"role": "system", "content": "너는 동화 생성 AI야."},
            {"role": "user", "content": story_start_prompt}]
story = ask_gpt(messages)

print("\n📖 [동화 시작]\n", story)
story_log.append("[동화 시작]\n" + story)

# ====== 동화 시작 그림 생성
generate_image(story, 1)



📖 [동화 시작]
 깊은 숲속 마을, 아침 햇살이 따사롭게 스며드는 날, 8살의 용사 도레미는 새로운 모험을 준비하고 있었습니다. 그녀는 커다란 모험가의 모자를 쓰고, 언제나 자신감을 가득 품고 있었지요. 정의로운 마음을 가진 도레미는 마을 사람들을 돕고 세상에 숨겨진 신비를 찾아가는 것을 좋아했습니다. 오늘도 그녀는 작은 가방에 필요한 물건들을 챙기고, 마을 경계 너머 미지의 세계로 발걸음을 내디딜 준비를 합니다. 무엇이 기다리고 있을지 알 수 없지만, 용감한 도레미는 그 모든 것을 기꺼이 마주할 각오가 되어 있었습니다.

🎨 [장면 1 프롬프트]
 anime style, masterpiece, pastel color, soft light, an 8-year-old courageous and righteous warrior named Doremi in a sunlit forest village, wearing a large adventurer's hat filled with confidence, preparing for a new adventure. She gathers necessary items in her small bag, ready to step beyond the village borders into the unknown world. Her expression is one of excitement and determination, with the morning sunlight gently illuminating her face and surroundings. The scene conveys a sense of magic and anticipation, capturing the essence of a brave young adventurer eager to discover hidden mysteries.
아이 그림을 업로드하세요.


Saving IMG_9610.JPG to IMG_9610.JPG


Token indices sequence length is longer than the specified maximum sequence length for this model (115 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['determination, with the morning sunlight gently illuminating her face and surroundings. the scene conveys a sense of magic and anticipation, capturing the essence of a brave young adventurer eager to discover hidden mysteries.']


  0%|          | 0/60 [00:00<?, ?it/s]

✅ 저장 완료 → storybook_scene_1.png


In [ ]:
# ====== 3. 아이 참여 + 동화 이어가기 (3회 → 장면 2,3,4 생성)
for step in range(3):

    # AI가 질문 + 이어질 동화 생성
    question_prompt = f"""
이전 동화 내용:

{story}

아이에게 물어볼 질문 1개와 이어질 동화 내용을 써줘.

[형식]
질문: (아이에게 던질 질문)
줄거리: (이야기 계속)
"""

    messages.append({"role": "user", "content": question_prompt})
    question_story = ask_gpt(messages)

    print(f"\n👦 [AI 질문 + 동화 예측 - Step {step+1}]\n", question_story)
    story_log.append(f"[Step {step+1}]\n" + question_story)

    user_answer = input("아이로서 대답하세요: ")

    continue_prompt = f"""
아이가 이렇게 대답했어: "{user_answer}"

이어서 자연스럽게 다음 줄거리를 써줘.

[동화 계속]
"""

    messages.append({"role": "user", "content": continue_prompt})
    story = ask_gpt(messages)

    print("\n📖 [동화 계속]\n", story)
    story_log.append(story)

    # 줄거리 기반 그림 생성
    generate_image(story, step + 2)

    time.sleep(1)


👦 [AI 질문 + 동화 예측 - Step 1]
 질문: 도레미가 가장 먼저 탐험하고 싶은 곳은 어디일까요?

줄거리: 도레미는 마을 경계를 넘어 나무가 빽빽한 숲 속으로 들어갔습니다. 그녀는 오래된 전설에 따르면 숲 깊은 곳에 숨겨진 빛나는 호수가 있다는 이야기를 기억하고 있었습니다. 호수를 찾기 위해서는 용감하게 나아가야 했지요. 길을 따라 걸으며, 도레미는 나무 사이로 작은 새들을 발견하고, 흙 속에서 반짝이는 돌멩이들을 집으며 가슴이 두근거렸습니다. 어떤 신비가 그녀를 기다리고 있을지 기대에 찬 도레미는 멀리서 나직하게 들려오는 물소리를 따라 발걸음을 재촉했습니다.
아이로서 대답하세요: 오두막집

📖 [동화 계속]
 질문: 용감한 도레미는 모험을 떠나기 전, 마을에서 어떤 특별한 장소를 방문하고 싶어 했을까요?

줄거리: 도레미는 모험을 떠나기 전, 마을 가장자리에 있는 작은 오두막집을 방문하기로 했습니다. 오두막집에는 마을의 현자인 할머니가 살고 계셨는데, 그녀는 항상 도레미에게 여러 가지 지혜를 나누어 주곤 했습니다. 도레미는 할머니에게 인사를 드리고, 모험을 위한 축복과 조언을 받기로 했습니다. 할머니는 부드러운 미소로 도레미를 맞이하며, 희미한 빛이 감도는 오두막 안에서 따뜻한 차를 건네고는 속삭였습니다. "마음속의 희망과 정의를 따라가거라. 너의 여정이 언제나 밝게 빛날 것이다." 도레미는 할머니의 말에 용기를 얻어, 더 단단한 마음으로 오두막을 나섰습니다.

🎨 [장면 2 프롬프트]
 Anime style, masterpiece, pastel color, soft light: In a quaint and serene forest village setting, young and brave adventurer Doremi, an 8-year-old courageous girl, stands confidently at the door of a small, cozy cabin at the village’s edge. She wears a large

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['bag strapped over her shoulder. an elderly wise woman welcomes her inside with a gentle smile, offering a cup of steaming tea. the interior of the hut is warm and inviting, bathed in a soft light that highlights the various trinkets and books of wisdom scattered around. doremi listens intently, absorbing the elder ’ s words of encouragement as she prepares to embark on her journey beyond the village boundaries.']


Saving JPEG 이미지-4FFD-BD81-FF-0.jpeg to JPEG 이미지-4FFD-BD81-FF-0.jpeg


  0%|          | 0/60 [00:00<?, ?it/s]

✅ 저장 완료 → storybook_scene_2.png

👦 [AI 질문 + 동화 예측 - Step 2]
 질문: 도레미가 오두막을 나선 후, 그녀의 모험에서 가장 먼저 마주친 것은 무엇일까요?

줄거리: 도레미가 오두막을 나서자마자, 길모퉁이에서 낯설고 신비한 작은 동물과 마주쳤습니다. 그 동물은 고운 파스텔 색의 털을 가진 작은 여우로, 꼭 동화 속에서 튀어나온 것처럼 보였습니다. 작은 여우는 도레미를 부드럽게 쳐다보더니, 그녀에게 다가와 길을 안내하는 듯 앞서 걷기 시작했습니다. 도레미는 호기심을 느끼며 여우를 따라가기로 마음먹었습니다. 어느새 여우는 도레미를 꽃이 만발한 비밀스러운 정원으로 안내했습니다.
아이로서 대답하세요: 토끼

📖 [동화 계속]
 
질문: 모험을 떠나는 도레미가 마주칠 첫 번째 친구는 누구였을까요?

줄거리: 오두막을 나와 깊은 숲으로 향하던 도레미는 작은 토끼 한 마리를 마주치게 되었습니다. 그 토끼는 도레미를 향해 반짝이는 눈으로 바라보며, 살며시 다가와서는 기대에 찬 몸짓으로 도레미 주위를 맴돌았습니다. 도레미는 토끼에게 다정한 미소를 지으며 인사를 했고, 토끼는 금방이라도 말을 걸 것처럼 깡충거렸습니다. "나와 함께 모험을 떠나겠니?" 도레미가 물었고, 토끼는 그저 기쁜 듯 깡충깡충 뛰며 답을 대신했습니다. 이렇게 도레미는 새로운 친구와 함께 모험의 첫걸음을 내딛게 되었습니다.

🎨 [장면 3 프롬프트]
 Anime style, masterpiece, pastel color, soft light; an 8-year-old brave and just adventurer girl named Doremi with a large explorer's hat, meeting a small, bright-eyed rabbit in a sun-dappled magical forest. The rabbit, seemingly full of excitement, hops around Doremi, who smiles warmly at it.

...

✅ 저장 완료 → storybook_scene_3.png

👦 [AI 질문 + 동화 예측 - Step 3]
 질문: 도레미와 토끼가 모험을 떠나면서 먼저 발견한 신비로운 장소는 어디였을까요?

줄거리: 도레미와 토끼는 숲길을 따라 걸어가다가 어느덧 울창한 나무들 사이로 빛나는 작은 연못을 발견했습니다. 연못은 마치 별들이 가득 담긴 하늘을 옮겨 놓은 듯 반짝였고, 그 주위로 꽃들이 활짝 피어 있었습니다. 도레미는 연못 가장자리에 앉아 아름다운 경치에 감탄하며, 토끼와 함께 연못의 물결을 바라보았습니다. 연못은 단순한 물이 아닌, 마치 무언가 특별한 의미를 지니고 있는 듯한 느낌이 들었습니다. 이 특별한 장소에서 휴식을 취한 도레미와 토끼는 마음이 한층 더 가벼워져, 앞으로의 모험에 대한 설렘을 안고 다시 길을 나섰습니다.
아이로서 대답하세요: 얼음성

📖 [동화 계속]
 
질문: 도레미와 토끼가 함께 찾은 첫 번째 모험의 장소는 어디였을까요?

줄거리: 도레미와 토끼는 신비한 숲 속을 지나 탁 트인 길에 이르렀고, 그곳에서 멀리 눈 덮인 산의 정상에 얼음성을 발견했습니다. 설렘에 가득 찬 도레미는 토끼와 함께 얼음성으로 향하기로 결심했습니다. 눈부신 햇살 아래 반짝이는 얼음성은 가까이 다가설수록 더욱 신비롭고 웅장하게 보였습니다. 어쩌면 그곳에선 마법 같은 이야기가 기다리고 있을지도 모른다는 생각에 두 친구는 발걸음을 재촉했습니다. 성으로 가는 길은 험난할지도 모르지만, 용감한 도레미와 함께라면 토끼는 두렵지 않았습니다.

🎨 [장면 4 프롬프트]
 Anime style, masterpiece, pastel color, soft light. A brave young girl named Doremi, wearing a large adventurer's hat, walks alongside a small, joyful rabbit. They journey towards a majestic ice castle perched atop a snow-covered mou

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['hinting at a magical and epic adventure ahead.']


Saving JPEG 이미지-48FB-9C1D-B3-0.jpeg to JPEG 이미지-48FB-9C1D-B3-0.jpeg


  0%|          | 0/60 [00:00<?, ?it/s]

✅ 저장 완료 → storybook_scene_4.png


In [ ]:
# ================================================
# ✅ 4. 전체 동화 정리
# ================================================

print("\n\n=== 📚 [전체 동화 정리] ===\n")
for part in story_log:
    print(part)



=== 📚 [전체 동화 정리] ===

[주인공 설정]

[주인공 캐릭터]
이름: 도레미
종류: 용사
나이: 8
성격: 용감한, 정의로운
특징: 모험을 좋아함

[동화 시작]
깊은 숲속 마을, 아침 햇살이 따사롭게 스며드는 날, 8살의 용사 도레미는 새로운 모험을 준비하고 있었습니다. 그녀는 커다란 모험가의 모자를 쓰고, 언제나 자신감을 가득 품고 있었지요. 정의로운 마음을 가진 도레미는 마을 사람들을 돕고 세상에 숨겨진 신비를 찾아가는 것을 좋아했습니다. 오늘도 그녀는 작은 가방에 필요한 물건들을 챙기고, 마을 경계 너머 미지의 세계로 발걸음을 내디딜 준비를 합니다. 무엇이 기다리고 있을지 알 수 없지만, 용감한 도레미는 그 모든 것을 기꺼이 마주할 각오가 되어 있었습니다.
[Step 1]
질문: 도레미가 가장 먼저 탐험하고 싶은 곳은 어디일까요?

줄거리: 도레미는 마을 경계를 넘어 나무가 빽빽한 숲 속으로 들어갔습니다. 그녀는 오래된 전설에 따르면 숲 깊은 곳에 숨겨진 빛나는 호수가 있다는 이야기를 기억하고 있었습니다. 호수를 찾기 위해서는 용감하게 나아가야 했지요. 길을 따라 걸으며, 도레미는 나무 사이로 작은 새들을 발견하고, 흙 속에서 반짝이는 돌멩이들을 집으며 가슴이 두근거렸습니다. 어떤 신비가 그녀를 기다리고 있을지 기대에 찬 도레미는 멀리서 나직하게 들려오는 물소리를 따라 발걸음을 재촉했습니다.
질문: 용감한 도레미는 모험을 떠나기 전, 마을에서 어떤 특별한 장소를 방문하고 싶어 했을까요?

줄거리: 도레미는 모험을 떠나기 전, 마을 가장자리에 있는 작은 오두막집을 방문하기로 했습니다. 오두막집에는 마을의 현자인 할머니가 살고 계셨는데, 그녀는 항상 도레미에게 여러 가지 지혜를 나누어 주곤 했습니다. 도레미는 할머니에게 인사를 드리고, 모험을 위한 축복과 조언을 받기로 했습니다. 할머니는 부드러운 미소로 도레미를 맞이하며, 희미한 빛이 감도는 오두막 안에서 따뜻한 차를 건네고는 속삭였습니다. "마음속의 희망과 정의를 따라가거라. 너의 여정이